In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 6 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

28 November 2022 ; 13:15:19


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,12,0,20.4,3.0,7.7,0.391,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
1,OG Anunoby,SF,25,TOR,19,19,36.6,7.1,15.3,0.466,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,0
2,Dalano Banton,PG,23,TOR,15,2,11.2,2.3,5.5,0.427,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
3,Scottie Barnes,PF,21,TOR,16,16,33.5,6.0,13.7,0.438,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,0
4,Khem Birch,C,30,TOR,9,0,8.0,1.0,1.6,0.643,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Brook Lopez,MIL,2.2,0.7,2.9,104,5.178947,2,0.270471
1,Nikola Jokić,DEN,3.7,1.4,0.6,111,3.587500,2,0.162404
2,Giannis Antetokounmpo,MIL,2.7,0.9,1.0,103,5.637500,2,0.137546
3,Chris Paul,PHO,2.2,1.7,0.2,109,4.100000,1,0.118913
4,Ivica Zubac,LAC,1.5,0.4,1.9,104,5.076190,5,0.113291


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)